Cofactor swap targets for oxygen

In [ ]:
%run 01_Model_CFE.ipynb


In [ ]:
import numpy as np
from tqdm import tqdm

from scipy.integrate import solve_ivp

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import cobra
from cobra.io import load_model
iMM904 = load_model('iMM904')

In [ ]:
def add_dynamic_bounds(iMM904, y):
    """Use external concentrations to bound the uptake flux of sucrose."""
    biomass, oxygen = y  # expand the boundary species
    oxygen_max_import = -10 * oxygen / (5 + oxygen)
    iMM904.reactions.EX_o2_e.lower_bound = oxygen_max_import


def dynamic_system(t, y):
    """Calculate the time derivative of external species."""

    biomass, oxygen = y  # expand the boundary species

    # Calculate the specific exchanges fluxes at the given external concentrations.
    with iMM904:
        add_dynamic_bounds(iMM904, y)

        cobra.util.add_lp_feasibility(iMM904)
        feasibility = cobra.util.fix_objective_as_constraint(iMM904)
        lex_constraints = cobra.util.add_lexicographic_constraints(
            iMM904, ['BIOMASS_SC5_notrace', 'EX_o2_e'], ['max', 'max'])
    fluxes = lex_constraints.values
    fluxes *= biomass
    if dynamic_system.pbar is not None:
        dynamic_system.pbar.update(1)
        dynamic_system.pbar.set_description('t = {:.3f}'.format(t))

    return fluxes


In [ ]:
dynamic_system.pbar = None

In [ ]:

def infeasible_event(t, y):
    with iMM904:

        add_dynamic_bounds(iMM904, y)

        cobra.util.add_lp_feasibility(iMM904)
        feasibility = cobra.util.fix_objective_as_constraint(iMM904)

    return feasibility - infeasible_event.epsilon

infeasible_event.epsilon = 1E-6
infeasible_event.direction = 1
infeasible_event.terminal = True

In [11]:
ts = np.linspace(0, 15, 100)  # Desired integration resolution and interval
y0 = [0.1, 10]

with tqdm() as pbar:
    dynamic_system.pbar = pbar

    sol = solve_ivp(
        fun=dynamic_system,
        events=[infeasible_event],
        t_span=(ts.min(), ts.max()),
        y0=y0,
        t_eval=ts,
        rtol=1e-6,
        atol=1e-8,
        method='BDF'
    )

    

In [ ]:
sol

In [ ]:
ax = plt.subplot(111)
ax.plot(sol.t, sol.y.T[:, 0])
ax2 = plt.twinx(ax)
ax2.plot(sol.t, sol.y.T[:, 1], color='r')

ax.set_ylabel('Biomass', color='b')
ax2.set_ylabel('Oxygen', color='r')
